In [ ]:
DROP TABLE IF EXISTS cities, weather, pollution, weather_pollution_facts;

In [ ]:
-- Dimension Table: Cities

CREATE TABLE cities (
    city_id SERIAL PRIMARY KEY,
    city_name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    population INTEGER NOT NULL,
    is_capital BOOLEAN
);

-- Dimension Table: Weather

CREATE TABLE weather (
    weather_id SERIAL PRIMARY KEY,
    city_id INTEGER NOT NULL,
    date_time TIMESTAMP NOT NULL,
    local_time TIMESTAMP NOT NULL,
    temperature FLOAT,
    feels_like FLOAT,
    temp_min FLOAT,
    temp_max FLOAT,
    pressure INTEGER,
    humidity INTEGER,
    visibility INTEGER,
    wind_speed FLOAT,
    wind_deg INTEGER,
    clouds_all INTEGER,
    weather_main VARCHAR(50),
    weather_description VARCHAR(100),
    weather_icon VARCHAR(10),
    sunrise TIMESTAMP,
    sunset TIMESTAMP,
    FOREIGN KEY (city_id) REFERENCES cities(city_id),
    UNIQUE (city_id, date_time) 
);


-- Dimension Table: Pollution

CREATE TABLE pollution (
    pollution_id SERIAL PRIMARY KEY,
    city_id INTEGER NOT NULL,
    date_time TIMESTAMP NOT NULL,
    local_time TIMESTAMP NOT NULL,
    aqi INTEGER,
    co FLOAT,
    no FLOAT,
    no2 FLOAT,
    o3 FLOAT,
    so2 FLOAT,
    pm2_5 FLOAT,
    pm10 FLOAT,
    nh3 FLOAT,
    FOREIGN KEY (city_id) REFERENCES cities(city_id),
    UNIQUE (city_id, date_time) 
);

-- Fact Table: Weather and Pollution Facts

CREATE TABLE weather_pollution_facts (
    fact_id SERIAL PRIMARY KEY,
    city_id INTEGER NOT NULL,
    date_time TIMESTAMP NOT NULL,
    local_time TIMESTAMP NOT NULL,
    weather_id INTEGER NOT NULL,
    pollution_id INTEGER NOT NULL,
    FOREIGN KEY (weather_id) REFERENCES weather(weather_id),
    FOREIGN KEY (pollution_id) REFERENCES pollution(pollution_id),
    UNIQUE (weather_id, pollution_id)  -- Ensure unique pairing of weather and pollution records
);



In [ ]:

COPY cities(city_name, latitude,longitude,country,population,is_capital,city_id) 
FROM 'D:\Datasets\Weather and Pollution\Data\Final\Final_city_data.csv' DELIMITER ',' CSV HEADER;

In [ ]:
-- Insert into fact table by linking weather and pollution records
INSERT INTO weather_pollution_facts (weather_id, pollution_id)
SELECT 
    w.weather_id,
    p.pollution_id
FROM weather w
JOIN pollution p ON w.city_id = p.city_id AND w.date_time = p.date_time
WHERE NOT EXISTS (
    SELECT 1 
    FROM weather_pollution_facts wf
    WHERE wf.weather_id = w.weather_id
    AND wf.pollution_id = p.pollution_id
);



In [ ]:
DELETE FROM weather;
DELETE FROM pollution;


In [ ]:
SELECT count(*) FROM weather;
SELECT count(*) FROM cities;
SELECT count(*) FROM pollution;